In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random

In [5]:
df = pd.read_csv('../../Data/final_cool_dataset.csv')
edges = [(row['ORIGIN'], row['DEST'], {'weight': row['PASSENGERS']}) for _, row in df.iterrows()]

In [59]:

class Simulation:
    
    def __init__(self, edgelist):
        self.Graph = nx.DiGraph()
        self.Graph.add_edges_from(edgelist)
        for node in self.Graph.nodes():
            self.Graph.nodes[node]['sum_weight_root'] = np.sqrt(sum([self.Graph[node][i]['weight'] for i in self.Graph.neighbors(node)]) + 0.001)

    def one_tick(self, beta, infected):
        '''
        simulates one tick, taking a beta value and a set of already infected nodes
        returns a new set of infected nodes
        '''
        
        add_to_set = set()
        
        for node in infected:

            for neighbor in self.Graph.neighbors(node):
                if neighbor not in infected:
                    if beta * ((self.Graph[node][neighbor]['weight'] + 1) / self.Graph.nodes[node]['sum_weight_root']) > random.random():
                        add_to_set.add(neighbor)
                    
        return infected | add_to_set
    
    def simulate_amount_of_ticks(self, beta, outset='random', ticks=1):
        '''
        simulates specified amount of ticks, taking a beta value, an amount of ticks 
        and an initial outset of infected nodes (defaults a random node)
        returns a list of sets, each element being the set of infected nodes for each tick
        '''
        
        if outset == 'random':
            infected_list = [{random.choice(list(self.Graph.nodes()))}]
        else:
            infected_list = [outset]
        
        for _ in range(ticks):
            infected_list.append(self.one_tick(beta, infected_list[-1]))
    
        return infected_list
    
    def simulate_until_threshold(self, beta, threshold, outset='random', limit=500):
        '''
        simulates until a certain number of nodes are infected, taking a beta value,
        a threshold and a limit for amount of ticks
        returns the amount of ticks to get to threshold
        '''
        
        if outset == 'random':
            infected = {random.choice(list(self.Graph.nodes()))}
        else:
            infected = outset
        
        for i in range(limit):
            infected = self.one_tick(beta, infected)
            
            if len(infected) >= threshold:
                return i
    
        return limit
    
    def threshold_trials(self, beta, threshold, outset='random', limit=500, trials=10):
        '''
        simulates a number of trials using the threshold method,
        returning a list of the amount of ticks to reach threshold
        '''
        
        return [self.simulate_until_threshold(beta=beta, threshold=threshold, outset=outset, limit=limit) for _ in range(trials)]   

In [60]:
c = Simulation(edges)

In [61]:
b = 0.005

In [62]:
def p_r(count,x):
    print(count)
    return x

In [63]:
test = [p_r(i,c.threshold_trials(b,250, trials=500)) for i in range(10)]

0
1
2
3
4
5
6
7
8
9


In [64]:
[sum(i)/len(i) for i in test]

[24.256, 21.482, 21.08, 22.556, 21.608, 21.864, 23.502, 22.188, 23.822, 22.056]

In [26]:
len(c.Graph.nodes())

1000